# Makemore - Backprop Ninja

## Setup and data

# Import packages

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

### Import data

In [2]:
words = open('../data/names.txt').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


### Build the vocabulary and map to/from integers

In [3]:
chars = sorted(list(set(''.join(words))))
itos = {i+1:s for i, s in enumerate(chars)}
itos[0] = '.'
stoi = {s:i for i, s in itos.items()}
vocab_size = len(itos)

print(vocab_size)
print(itos)

27
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


### Build the dataset

In [4]:
block_size = 3

def build_dataset(words):
    X, Y = [], []
    for name in words:
        context = [0] * block_size
        for char in name + '.':
            ix = stoi[char]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xtst, Ytst = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


## Neural Network

### Utility function

In [5]:
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt- t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

### Neural Network initialisation

In [6]:
n_emb = 10
n_hidden = 64
g = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size, n_emb),                  generator=g)
# Layer 1
W1 = torch.randn(((block_size * n_emb), n_hidden),     generator=g) * (5/3) / ((n_emb * block_size)**0.5)
b1 = torch.randn(n_hidden,                            generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),              generator=g) * 0.1
b2 = torch.randn(vocab_size,                          generator=g) * 0.1

bngain = torch.randn((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.randn((1, n_hidden)) * 0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

4137


In [7]:
batch_size = 32
n = batch_size

ix = torch.randint(0, Xtr.shape[0], (n,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

### Training layer

In [8]:
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)

# Linear layer 1
hprenb = embcat @ W1 + b1

# BatchNorm layer
# - Calculate the variance and stuff
bnmeani = 1 / n * hprenb.sum(0, keepdim=True)
bndiff = hprenb - bnmeani
bndiff2 = bndiff**2
bnvar = 1 / (n - 1) * (bndiff2).sum(0, keepdim=True)

# - Normalization
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv

# - Scale and shift
hpreact = bngain * bnraw + bnbias

# Non-linearity layer
h = torch.tanh(hpreact)

# Linear layer 2
logits = h @ W2 + b2

# Cross Entropy loss
logit_maxes = logits.max(1, keepdim=True).values # Get max values from each row
norm_logits = logits - logit_maxes # Normalize the values (everything is below zero)
counts = norm_logits.exp() # Softmax: Get the exponents (all between 0 and 1)
counts_sum = counts.sum(1, keepdim=True) # Softmax: Sum of all counts
counts_sum_inv = counts_sum**-1 # Softmax: inverse so we can divide with it
probs = counts * counts_sum_inv # Softmax: Divide the exponent of each count by the sum of exponents of all counts
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backwards pass
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
          bnvar_inv, bnvar, bndiff2, bndiff, hprenb, bnmeani,
          embcat, emb]:
    t.retain_grad()
loss.backward()
loss

tensor(3.3295, grad_fn=<NegBackward0>)

In [21]:
counts.shape, counts_sum.shape, counts_sum_inv.shape

(torch.Size([32, 27]), torch.Size([32, 1]), torch.Size([32, 1]))

In [18]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1 / n
dprobs = (1 / probs) * dlogprobs
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)
dcounts_sum = -1 / counts_sum**2 * dcounts_sum_inv
dcounts = (torch.ones_like(* dcounts_sun_inv)

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
# cmp('counts', dcounts, counts)
# cmp('norm_logits', dnorm_logits, norm_logits)
# cmp('logit_maxes', dlogit_maxes, logit_maxes)
# cmp('logits', dlogits, logits)
# cmp('h', dh, h)
# cmp('W2', dW2, W2)
# cmp('b2', db2, b2)
# cmp('hpreact', dhpreact, hpreact)
# cmp('bngain', dbngain, bngain)
# cmp('bnbias', dbnbias, bnbias)
# cmp('bnraw', dbnraw, bnraw)
# cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
# cmp('bnvar', dbnvar, bnvar)
# cmp('bndiff2', dbndiff2, bndiff2)
# cmp('bndiff', dbndiff, bndiff)
# cmp('bnmeani', dbnmeani, bnmeani)
# cmp('hprebn', dhprebn, hprebn)
# cmp('embcat', dembcat, embcat)
# cmp('W1', dW1, W1)
# cmp('b1', db1, b1)
# cmp('emb', demb, emb)
# cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
